In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from main import *

from sklearn.cluster import KMeans
from datasetUtils import load_from_Jadson
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


# if __name__ == '__main__':
# parser = argparse.ArgumentParser(description='Define the UDA parameters')
#
# parser.add_argument('--gpu_ids', type=str, default="7", help='GPU IDs')
# parser.add_argument('--lr', type=float, default=3.5e-4, help='Learning Rate')
# parser.add_argument('--P', type=int, default=16, help='Number of Persons')
# parser.add_argument('--K', type=int, default=4, help='Number of samples per person')
# parser.add_argument('--tau', type=float, default=0.05, help='tau value used on softmax triplet loss')
# parser.add_argument('--beta', type=float, default=0.999, help='beta used on self-Ensembling')
# parser.add_argument('--k1', type=int, default=30, help='k on k-Reciprocal Encoding')
# parser.add_argument('--sampling', type=str, default="mean", help='Mean or Random feature vectors to be prototype')
# parser.add_argument('--lambda_hard', type=float, default=0.5, help='tuning prameter of Softmax Triplet Loss')
# parser.add_argument('--num_iter', type=int, default=400, help='Number of iterations on an epoch')
# parser.add_argument('--momentum_on_feature_extraction', type=int, default=0,
# help='If it is the momentum used on feature extraction')
# parser.add_argument('--target', type=str, help='Name of target dataset')
# parser.add_argument('--path_to_save_models', type=str, help='Path to save models')
# parser.add_argument('--path_to_save_metrics', type=str, help='Path to save metrics (mAP, CMC, ...)')
# parser.add_argument('--version', type=str, help='Path to save models')
# parser.add_argument('--eval_freq', type=int, help='Evaluation Frequency along training')

# args = parser.parse_args()
# gpu_ids = args.gpu_ids
# base_lr = args.lr
# P = args.P
# K = args.K

# tau = args.tau
# beta = args.beta
# k1 = args.k1
# sampling  = args.sampling
#
# lambda_hard = args.lambda_hard
# number_of_iterations = args.num_iter
# momentum_on_feature_extraction = bool(args.momentum_on_feature_extraction)
# target = args.target
# dir_to_save = args.path_to_save_models
# dir_to_save_metrics = args.path_to_save_metrics
# version = args.version
# eval_freq = args.eval_freq
# main.py --gpu_ids=0,1,2,3 --lr=3.5e-4 --P=16 --K=12 --tau=0.04 --beta=0.999 --k1=30 --sampling=mean --lambda_hard=0.5 --num_iter=7 --momentum_on_feature_extraction=0 --target=Duke --path_to_save_models=models --path_to_save_metrics=metrics --version=version_name --eval_freq=5

import sys
import os
import pandas as pd

from IPython.display import display, Image

from IPython.display import display, HTML
from bs4 import BeautifulSoup


# Função para exibir a imagem usando HTML
def exibir_imagem(imagem_path):
    return f'<img src="{imagem_path}" width="40">'


from metricas import *

html_content= ""
df = pd.DataFrame({
    'k':[], 
    'lambda_hard':[],
    'modelo':[],
    'matriz_confusao':[], 
    'Acuracia':[], 
    'Precisao':[],
    'Recall':[],
    'F1-score':[],
    'Grafico':[],
    'Tipo':[]
    })

gpus = "1,2,3"
for k in [4]:
    for lambda_hard in [ 0.0 ]:
                
        print(f"**** inicio do teste com ruido em k:{k} e lambda_hard:{lambda_hard} ****")
        
        version = f"teste-05-hsv-90epocas{k}_{lambda_hard}"
        
        main(gpu_ids=gpus,base_lr=3.5e-4,P=16,K=k,tau=0.04,beta=0.999,k1=30,sampling="random",lambda_hard=lambda_hard,number_of_iterations=7,momentum_on_feature_extraction=0,target="Jadson",dir_to_save="models",dir_to_save_metrics="metrics",version=version,eval_freq=5,use_ruido=True)
        
        for metodo in models_name + ["mean"]:
            metricas_t, metricas_v, rotulos_t, rotulos_v = metricas(k=k, lambda_hard=lambda_hard, modelo=metodo)
            linha = {
                'k':            [k], 
                'lambda_hard':  [lambda_hard],
                'modelo':       [metodo],
                'Tipo':         'Test'
            }
            for count in range( 0, metricas_t.shape[0] ):
                for m in range( 0, metricas_t.shape[1] ):
                    linha[rotulos_t[m]] = metricas_t[count][m]
                    
                
                linha['matriz_confusao'] = f'resultados/MC_{k}_{lambda_hard}_{count}_{metodo}_test.png'
                linha['Grafico'] = f'resultados/grafico_{k}_{lambda_hard}_{count}_{metodo}_test.png'
                df = pd.concat( [df, pd.DataFrame(linha)], axis=0)
             
            linha = {
               'k':             [k], 
               'lambda_hard':   [lambda_hard],
               'modelo':        [metodo],
               'Tipo':          'Valid'
             }
            for count in range( 0, metricas_v.shape[0] ):
                for m in range( 0, metricas_v.shape[1] ):
                    linha[rotulos_v[m]] = metricas_v[count][m] 
               
                linha['Grafico'] = f'resultados/grafico_{k}_{lambda_hard}_{count}_{metodo}_valid.png'
                linha['matriz_confusao'] = f'resultados/MC_{k}_{lambda_hard}_{count}_{metodo}_valid.png' 
                df = pd.concat( [df, pd.DataFrame(linha)], axis=0)
        
        # Aplicar a função à coluna 'imagem' e criar uma nova coluna 'imagem_exibicao'
        df['MC'] = df['matriz_confusao'].apply(exibir_imagem)
        df['GR'] = df['Grafico'].apply(exibir_imagem)
        
        html_content = df[['k', 
                           'lambda_hard', 
                           'Tipo', 
                           'modelo'] + 
                           rotulos_v[:8] + 
                           ['MC', 
                           'GR']].to_html(escape=False, index=False)
        # salvando df em arquivo html
        # Use BeautifulSoup para formatar o HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        formatted_html = soup.prettify()
        
        # Salve o HTML em um arquivo
        head = "<!DOCTYPE html>\n<html lang='pt-br'>\n<head>\n  <meta charset='UTF-8'>\n  <meta name='viewport' content='width=device-width, initial-scale=1.0'>\n  <style>\n    table {\n      width: 100%;\n      border-collapse: collapse;\n    }\n    th, td {\n      border: 1px solid #ddd;\n      padding: 8px;\n      text-align: left;\n    }\n    th {\n      background-color: #f2f2f2;\n    }\n    thead th {\n      position: sticky;\n      top: 0;\n      z-index: 1;\n      background-color: #f2f2f2;    }\n  </style>\n    <title>Relatório Parcial</title>\n</head>\n<body>"
        with open('relatorio-APCER-BPCER-ACER-silhouette-ward-90epocas-hsv-MNETv3-convnet-efficientnet.html', 'w', encoding='utf-8') as file:
            file.write(head)
            file.write(formatted_html)
            file.write('</body></html>')

/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly

**** inicio do teste com ruido em k:4 e lambda_hard:0.0 ****
Num GPU's: 3
Allocated GPU's for model: [1, 2]


/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Base_Weights.I

Successfully loaded imagenet pretrained weights from "/home/emorais/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
Successfully loaded imagenet pretrained weights from "/home/emorais/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"


/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


...

Jaccard distance computing time cost: 79.9932074546814
Extracting Online Features for osnet ...
Features extracted in 198.57 seconds
Computing jaccard distance...


bruteForceKnn is deprecated; call bfKnn instead


Jaccard distance computing time cost: 78.70398926734924
Extracting Online Features for densenet121 ...
Features extracted in 194.81 seconds
Computing jaccard distance...


bruteForceKnn is deprecated; call bfKnn instead


Jaccard distance computing time cost: 79.93593215942383
Reliability: 0.998
Mean Purity: 0.00164
There are 2 clusters with 4 cameras
There are 4 clusters with 5 cameras
There are 1 clusters with 8 cameras
There are 1 clusters with 10 cameras
There are 1 clusters with 13 cameras
There are 1 clusters with 16 cameras
There are 2 clusters with 18 cameras
There are 1 clusters with 26 cameras
There are 1 clusters with 27 cameras
There are 1 clusters with 30 cameras
There are 1 clusters with 34 cameras
There are 1 clusters with 37 cameras
There are 1 clusters with 38 cameras
There are 1 clusters with 44 cameras
There are 1 clusters with 47 cameras
There are 1 clusters with 48 cameras
There are 1 clusters with 52 cameras
There are 1 clusters with 54 cameras
There are 2 clusters with 58 cameras
There are 4 clusters with 59 cameras
There are 1 clusters with 60 cameras
There are 13 clusters with 61 cameras
There are 10 clusters with 62 cameras
There are 16 clusters with 63 cameras
There are 538 cl

In [2]:
# Exibir o DataFrame com as imagens
display(HTML(html_content))
print(df)

k,lambda_hard,Tipo,modelo,ACCURACY,PRECISION,RECALL,F1_SCORE,APCER,BPCER,ACER,SILHOUETTE,MC,GR
4.0,0.0,Test,efficientnet,0.588583,0.778854,0.678333,0.725127,0.321667,0.770417,0.546042,0.569139,,
4.0,0.0,Valid,efficientnet,0.697604,0.801084,0.827474,0.814065,0.172526,0.821875,0.497201,0.642409,,
4.0,0.0,Test,convnext,0.352833,0.757151,0.281250,0.410147,0.718750,0.360833,0.539792,0.653967,,
4.0,0.0,Valid,convnext,0.697917,0.797856,0.833594,0.815334,0.166406,0.844792,0.505599,0.644618,,
4.0,0.0,Test,mobilenet,0.534750,0.745328,0.635625,0.686119,0.364375,0.868750,0.616563,0.590126,,
4.0,0.0,Valid,mobilenet,0.676875,0.819782,0.764062,0.790942,0.235937,0.671875,0.453906,0.589765,,
4.0,0.0,Test,vgg16,0.410667,0.676364,0.504948,0.578219,0.495052,0.966458,0.730755,0.589213,,
4.0,0.0,Valid,vgg16,0.701146,0.798709,0.837500,0.817644,0.162500,0.844271,0.503385,0.595256,,
4.0,0.0,Test,resnet50,0.407458,0.676298,0.497396,0.573212,0.502604,0.952292,0.727448,0.602894,,
4.0,0.0,Valid,resnet50,0.425208,0.706376,0.481771,0.572844,0.518229,0.801042,0.659635,0.562353,,


     k  lambda_hard        modelo  \
0  4.0          0.0  efficientnet   
0  4.0          0.0  efficientnet   
0  4.0          0.0      convnext   
0  4.0          0.0      convnext   
0  4.0          0.0     mobilenet   
0  4.0          0.0     mobilenet   
0  4.0          0.0         vgg16   
0  4.0          0.0         vgg16   
0  4.0          0.0      resnet50   
0  4.0          0.0      resnet50   
0  4.0          0.0         osnet   
0  4.0          0.0         osnet   
0  4.0          0.0   densenet121   
0  4.0          0.0   densenet121   
0  4.0          0.0          mean   
0  4.0          0.0          mean   

                                matriz_confusao  Acuracia  Precisao  Recall  \
0   resultados/MC_4_0.0_0_efficientnet_test.png       NaN       NaN     NaN   
0  resultados/MC_4_0.0_0_efficientnet_valid.png       NaN       NaN     NaN   
0       resultados/MC_4_0.0_0_convnext_test.png       NaN       NaN     NaN   
0      resultados/MC_4_0.0_0_convnext_valid.png       